In [1]:
# Import libraries
import sys
sys.path.append('..')
from src.model import build_segformer3d_model, SegFormer3D
from src.preprocessing import get_transforms, get_datasets, get_dataloaders
import torch
import matplotlib.pyplot as plt
from src.train_validate import train, validate
from tqdm import tqdm

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ldeben20 (luciano-deben). Use `wandb login --relogin` to force relogin


cuda:0


In [2]:
# Get transforms
transform = get_transforms(resize_shape=[240, 240, 155], contrast_value=1000)

# Get datasets
train_ds, val_ds = get_datasets(
    root_dir="../data",
    collection="HCC-TACE-Seg",
    transform=transform,
    download_len=2,
    val_frac=0.2,
)

# Get dataloaders
train_loader, val_loader = get_dataloaders(train_ds, val_ds, batch_size=2)

# Get datalists
train_dl = train_ds.datalist
val_dl = val_ds.datalist

# Check length of datasets, dataloaders and datalists
print(train_ds.get_indices(), val_ds.get_indices())
print(len(train_loader), len(val_loader))
print(len(train_dl), len(val_dl))

2024-05-03 11:19:12,821 - INFO - Expected md5 is None, skip md5 check for file ..\data\HCC-TACE-Seg\1.2.276.0.7230010.3.1.3.8323329.41.1604860085.518229.zip.
2024-05-03 11:19:12,821 - INFO - File exists: ..\data\HCC-TACE-Seg\1.2.276.0.7230010.3.1.3.8323329.41.1604860085.518229.zip, skipped downloading.
2024-05-03 11:19:12,824 - INFO - Writing into directory: ..\data\HCC-TACE-Seg\raw\1.2.276.0.7230010.3.1.3.8323329.41.1604860085.518229.
2024-05-03 11:19:13,309 - INFO - Expected md5 is None, skip md5 check for file ..\data\HCC-TACE-Seg\1.3.6.1.4.1.14519.5.2.1.1706.8374.172517341095680731665822868712.zip.
2024-05-03 11:19:13,309 - INFO - File exists: ..\data\HCC-TACE-Seg\1.3.6.1.4.1.14519.5.2.1.1706.8374.172517341095680731665822868712.zip, skipped downloading.
2024-05-03 11:19:13,309 - INFO - Writing into directory: ..\data\HCC-TACE-Seg\HCC_017\300\image.
2024-05-03 11:19:13,568 - INFO - Expected md5 is None, skip md5 check for file ..\data\HCC-TACE-Seg\1.2.276.0.7230010.3.1.3.8323329.208

In [3]:
for inputs in tqdm(train_loader):
    images, targets = inputs["image"], inputs["seg"]

100%|██████████| 2/2 [00:15<00:00,  7.80s/it]


In [4]:
images, targets = images.to(device), targets.to(device)
model = SegFormer3D()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [5]:
model = model.to(device)
model.train()
# optimizer.zero_grad()

c1, c2, c3, c4 = model(images)
print(c1)
# loss = criterion(outputs, targets)
# loss.backward()
# optimizer.step()

# print(loss.item())

RuntimeError: shape '[2, 32, 52, 52, 52]' is invalid for input of size 8985600